In [41]:
import matplotlib.pyplot as plt
from countryinfo import CountryInfo
import psycopg2
import pandas as pd
import logging
import json
import os
from dotenv import load_dotenv
import pycountry
import time



In [2]:
load_dotenv('.env')
user = os.getenv("user")
password = os.getenv("password")
host = os.getenv("host")
port = os.getenv("port")
database = os.getenv("database")
SAVE_PATH = os.getenv("SAVE_PATH")

In [3]:

#SELECTS ALL JOBS
"""
def all_data_postgre() -> list :
	conn = psycopg2.connect(user=user, password=password, host=host, port=port, database=database)

	# Create a cursor object
	cur = conn.cursor()

	# Fetch new data from the table where id is greater than max_id
	cur.execute("SELECT * FROM main_jobs")
	data = cur.fetchall()

	# Fetch column names from the cursor's description
	column_names = [desc[0] for desc in cur.description]

	# Create a DataFrame with fetched data and set column names
	df = pd.DataFrame(data, columns=column_names)

	# Close the database connection
	conn.commit()
	cur.close()
	conn.close()

	return df

df = all_data_postgre()"""


'def all_data_postgre() -> list :\n\tconn = psycopg2.connect(user=user, password=password, host=host, port=port, database=database)\n\n\t# Create a cursor object\n\tcur = conn.cursor()\n\n\t# Fetch new data from the table where id is greater than max_id\n\tcur.execute("SELECT * FROM main_jobs")\n\tdata = cur.fetchall()\n\n\t# Fetch column names from the cursor\'s description\n\tcolumn_names = [desc[0] for desc in cur.description]\n\n\t# Create a DataFrame with fetched data and set column names\n\tdf = pd.DataFrame(data, columns=column_names)\n\n\t# Close the database connection\n\tconn.commit()\n\tcur.close()\n\tconn.close()\n\n\treturn df\n\ndf = all_data_postgre()'

In [4]:
# Plot unique entries
"""
num_unique_entries = df['location'].nunique()
print(f"Number of unique entries: {num_unique_entries}")

# Get the counts of each unique entry
counts = df['location'].value_counts()

# Plot the counts in a pie chart
plt.figure(figsize=(20, 8))
counts.plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribution of Locations')
plt.ylabel('')
plt.show()"""

'# Get the number of unique entries\nnum_unique_entries = df[\'location\'].nunique()\nprint(f"Number of unique entries: {num_unique_entries}")\n\n# Get the counts of each unique entry\ncounts = df[\'location\'].value_counts()\n\n# Plot the counts in a pie chart\nplt.figure(figsize=(20, 8))\ncounts.plot(kind=\'pie\', autopct=\'%1.1f%%\')\nplt.title(\'Distribution of Locations\')\nplt.ylabel(\'\')\nplt.show()'

In [31]:

#Subdivision per country code

with open(SAVE_PATH + '/country_abbreviations.json', 'r') as f:
    country_abbreviations = json.load(f)

countries = []
for country, abbreviation in country_abbreviations.items():
    country_dict = {"country_name": country, "country_code": abbreviation}
    try:
        # Get the subdivisions of the country
        time.sleep(.5)
        subdivisions = list(pycountry.subdivisions.get(country_code=abbreviation))
        if subdivisions:
            subdivisions_list = []
            for subdivision in subdivisions:
                subdivisions_list.append({"subdivisions_code": subdivision.code, "subdivisions_name": subdivision.name})
            country_dict["subdivisions"] = subdivisions_list
        else:
            country_dict["subdivisions"] = "NaN"
    except KeyError:
        country_dict["subdivisions"] = "NaN"
    countries.append(country_dict)

# Save the list of countries as a JSON file
with open(SAVE_PATH + '/country_subdivisions.json', 'w') as json_file:
    json.dump({"Countries": countries}, json_file, indent=4)

print("JSON FILE SAVED CORRECTLY")

JSON FILE SAVED CORRECTLY


In [38]:
#Continents [Countries[info]]

import json
from pycountry_convert import country_alpha2_to_continent_code, convert_continent_code_to_continent_name

# Load your existing JSON data
with open(SAVE_PATH + '/country_subdivisions.json', 'r') as f:
    data = json.load(f)

# Initialize an empty dictionary to hold the new data structure
continent_countries = {}

# Iterate over the countries in your data
for country in data['Countries']:
    # Get the country code
    country_code = country['country_code']

    # Get the continent code and convert it to continent name
    try:
        continent_code = country_alpha2_to_continent_code(country_code)
        continent_name = convert_continent_code_to_continent_name(continent_code)
    except KeyError:
        # If the country code is not recognized, skip this country
        continue

    # If the continent is not yet in the dictionary, add it
    if continent_name not in continent_countries:
        continent_countries[continent_name] = {'Countries': []}

    # Add the country to the continent's list of countries
    continent_countries[continent_name]['Countries'].append(country)

# Save the new data structure as a JSON file
with open(SAVE_PATH + '/continent_countries.json', 'w') as f:
    json.dump(continent_countries, f, indent=4)

In [43]:

#Get the capital cities in english

with open(SAVE_PATH + '/continent_countries.json', 'r') as f:
    data = json.load(f)

# Iterate over the countries in your data
for continent in data.values():
    for i, country in enumerate(continent['Countries']):
        # Get the country name
        country_name = country['country_name']

        # Get the capital city
        try:
            country_info = CountryInfo(country_name)
            capital_city = country_info.capital()
        except KeyError:
            # If the country name is not recognized, set the capital to 'NaN'
            capital_city = 'NaN'

        # Create a new dictionary for the country with the keys in the desired order
        new_country = {
            'country_name': country_name,
            'country_code': country['country_code'],
            'capital_english': capital_city,
            'subdivisions': country['subdivisions']
        }

        # Replace the old country dictionary with the new one
        continent['Countries'][i] = new_country

# Save the new data structure as a JSON file
with open(SAVE_PATH + '/continent_countries_with_capitals.json', 'w') as f:
    json.dump(data, f, indent=4)

In [52]:
def all_location_postgre() -> list :
	conn = psycopg2.connect(user=user, password=password, host=host, port=port, database=database)

	# Create a cursor object
	cur = conn.cursor()

	# Fetch new data from the table where id is greater than max_id
	#cur.execute("SELECT DISTINCT location FROM main_jobs")
	cur.execute("SELECT DISTINCT location FROM main_jobs")

	data = cur.fetchall()

	# Create a DataFrame with fetched data and set column names
	df = pd.DataFrame(data, columns=["location"])

	# Close the database connection
	cur.close()
	conn.close()

	return df

df = all_location_postgre()

df.to_csv(SAVE_PATH + '/location_distinct.csv', index=False)


In [53]:


def country_name_to_code():
    # Country name transformed to their respective codes & clean
    with open(SAVE_PATH + '/continent_countries_with_capitals.json', 'r') as f:
        data = json.load(f)

    # Create dictionaries for country names and subdivision names
    country_dict = {}
    subdivision_dict = {}
    for continent in data.values():
        for country in continent['Countries']:
            country_dict[country['country_name']] = country['country_code']

    def convert_names_to_codes(row, country_dict=country_dict, subdivision_dict=subdivision_dict):
        # Split the string into a list of names
        names = row.split()

        # Replace each name with its code
        codes = [country_dict.get(name, name) for name in names]

        # Remove duplicates and join the codes back into a string
        return ' '.join(pd.unique(codes))

    # Apply the function to the 'location' column
    df['location'] = df['location'].apply(convert_names_to_codes)

    df.to_csv(SAVE_PATH + '/location_distinct.csv', index=False)

In [54]:
# Split the 'countries' column into separate country codes
country_codes = df['location'].str.split()

# Flatten the list of lists into a single list
country_codes = [code for sublist in country_codes for code in sublist]

# Convert the list into a Series and count the occurrences of each country code
country_counts = pd.Series(country_codes).value_counts()

# Convert the Series to a DataFrame
country_counts_df = country_counts.reset_index()

# Rename the columns
country_counts_df.columns = ['Country Code', 'Count']

# Print the DataFrame
print(country_counts_df)

country_counts_df.to_csv(SAVE_PATH + '/location_distinct_counts.csv', index=False)


     Country Code  Count
0          Remote    379
1              US    160
2              CA    115
3              UK     49
4          United     45
..            ...    ...
246        states      1
247  Remote-Phnom      1
248     Northeast      1
249           UK-      1
250            JO      1

[251 rows x 2 columns]
